# Lab 9: Reinforcement Learning

- Saranpat Funkaew, 58070501068
- Chatthong Rimthong, 58070501011

ในแลปนี้จะเป็นการสร้าง Model ให้สามารถเรียนรู้ด้วยตัวเองได้โดยไม่มี คำตอบ (Label) ให้ในการ train แต่จะเป็นการใช้รางวัล (Reward) ให้กับตัว Model แทน หรือเรียกว่า Reinforcement Learning 

ซึ่งจะมี Reference ของการทำแลปนี้ https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c?fbclid=IwAR3grzzDBY1ZDTfeydV7VEqH9s-pFZCqwQlES5xm4uHGcZzEnVZJMXnA8eQ 
ในเว็บไซด์เขาได้ทำการสร้าง Model มาเพื่อเล่นเกม "MountainCar-v0" ซึ่งเป็นเกมสำหรับฝึก AI โดยจะสร้างเกมจาก library gym และใน gym ก็ยังมีอีกหลายเกมให้ลองเล่นเช่นกัน 

In [2]:
# Import required libraries
import gym
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from collections import deque

Using TensorFlow backend.


## DQN

เริ่มจากสร้าง class DQN(Deep Q Network) หรือก็คือ Agent สำหรับการแก้ปัญหานั้น ๆ ใน Environment โดยใน class จะประกอบด้วย variable 10 ตัว<br>
1. env: หรือก็คือ Environment (ปัญหาที่ต้องการแก้) ซึ่งในที่นี้ก็คือ การทำให้รถวิ่งขึ้นไปสู่เส้นชัยให้ได้ (MountainCar-v0)
2. memory: คือตัวแปรสำหรับเก็บว่าในแต่ละครั้งที่ทดลองนั้นมีผลเป็นอย่างไร (state, action, reward, new_state, done)
3. gamma
4. epsilon
5. epsilon_min
6. epsilon_decay
7. learning_rate
8. tau
9. model: สำหรับ predict ว่าจะต้องทำ Action อะไร
10. target_model: สำหรับ predict ว่า Model นั้นจะต้อง Take Action อะไร

*ที่จำเป็นต้องใช้ 2 Model เพราะถ้าใช้ Model เพียงอย่าเดียวจะทำให้ดป้าหมายนั้นเปลี่ยนไปเรื่อย ๆ  จึงต้องใช้ 2 Model เพื่อให้เป้าหมายนั้นคงที่เป็นเหมือนเดิม

ในส่วนของ Function จะมี
1. create_model: สำหรับสร้าง Model หรือกำหนดว่า network นั้นมี layer อะไรบ้าง
2. act: สำหรับทำอะไรบางอย่างกับ Env ซึ่งในที่นี้คือการ predict ว่า Action ต่อไปควรจะเป็นอย่างไร
3. remember: คือการจำค่าที่ input และ output ออกมาจาก env 
4. replay: สำหรับการ train model
5. target_train: สำหรับการ train target_model
6. save_model: Save weight

In [3]:
class DQN:
    def __init__(self, env):
        self.env     = env
        self.memory  = deque(maxlen=2000)
        
        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.005
        self.tau = .125

        self.model        = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        state_shape  = self.env.observation_space.shape
        model.add(Dense(24, input_dim=state_shape[0], activation="relu"))
        model.add(Dense(48, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(self.env.action_space.n))
        model.compile(loss="mean_squared_error",
            optimizer=Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size = 32
        if len(self.memory) < batch_size: 
            return

        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)

### สร้าง Environment

In [4]:
env = gym.make("MountainCar-v0")
trials  = 1000
trial_len = 500

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### สร้าง Agent

In [5]:
dqn_agent = DQN(env=env)

### เริ่มการ Training

โดยในการเทรนถ้า np.random.random() นั้นน้อยกว่าค่า epsilon จะเป็นค่าที่สุ่ม Action จาก function(env.action_space.sample()) ออกมา แต่ถ้ามากกว่าก็จะนำค่าจากที่ model predict มาเป็น Action เพื่อ Train Model ต่อไป

In [6]:
for trial in range(trials):
    #Reset Environpment(initial)
    cur_state = env.reset().reshape(1,2)
    for step in range(trial_len):
        # env.render(mode = 'rgb_array') # render how it play the game
        action = dqn_agent.act(cur_state) # model predict action 
        new_state, reward, done, _ = env.step(action) # send action to env

        # reward = reward if not done else -20
        new_state = new_state.reshape(1,2)
        dqn_agent.remember(cur_state, action, reward, new_state, done) # Remember input and output

        dqn_agent.replay()       # Train model 
        dqn_agent.target_train() # Train target_model

        cur_state = new_state
        if done:
            break
    if step >= 199:
        print("Failed to complete in trial {}".format(trial))
        if step % 10 == 0:
            dqn_agent.save_model("trial-{}.model".format(trial))
    else:
        print("Completed in {} trials".format(trial))
        dqn_agent.save_model("success.model")
        break

Failed to complete in trial 0
Failed to complete in trial 1
Failed to complete in trial 2
Failed to complete in trial 3
Failed to complete in trial 4
Failed to complete in trial 5
Failed to complete in trial 6
Failed to complete in trial 7
Failed to complete in trial 8
Failed to complete in trial 9
Failed to complete in trial 10
Failed to complete in trial 11
Failed to complete in trial 12
Failed to complete in trial 13
Failed to complete in trial 14
Failed to complete in trial 15
Failed to complete in trial 16
Failed to complete in trial 17
Failed to complete in trial 18
Failed to complete in trial 19
Failed to complete in trial 20
Failed to complete in trial 21
Failed to complete in trial 22
Failed to complete in trial 23
Failed to complete in trial 24
Failed to complete in trial 25
Failed to complete in trial 26
Failed to complete in trial 27
Failed to complete in trial 28
Failed to complete in trial 29
Failed to complete in trial 30
Failed to complete in trial 31
Failed to complete

ซึ่งจากในการ train จะเป็นการ random ทำให้ trial ของการ train แต่ละครั้งนั้นไม่เท่ากันขึ้นอยู่กับว่า random ได้ออกมาดีแค่ไหน